In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pickle

In [113]:
# load the model and transformers/encoders from disk
lm = pickle.load(open('fifa_model.sav', 'rb'))
transformer = pickle.load(open('fifa_transformer.sav', 'rb'))
bp_encoder = pickle.load(open('bp_encoder.sav', 'rb'))
foot_encoder = pickle.load(open('foot_encoder.sav', 'rb'))

In [72]:
# load the data
data = pd.read_csv('fifa_new_data.csv')
data.drop('Unnamed: 0', inplace=True, axis=1)
          
train_data = pd.read_csv('fifa21_training.csv')
train_data.drop('Unnamed: 0', inplace=True, axis=1)

In [73]:
#remove underscore from column names
data.columns = new_data.columns.str.replace("_", " ")

In [74]:
#can't find    'growth',  'base stats', 'hits',

In [75]:
data.columns

Index(['sofifa id', 'player url', 'short name', 'long name', 'age', 'dob',
       'height cm', 'weight kg', 'nationality', 'club',
       ...
       'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'],
      dtype='object', length=103)

In [76]:
to_drop = ['sofifa id', 'player url', 'short name', 'long name', 'dob', 'nationality', 'club',
           'release clause eur', 'team jersey number', 'loaned from', 'joined',
          'contract valid until', 'nation position', 'nation jersey number',
          'attacking crossing', 'attacking finishing', 'attacking heading accuracy',
           'attacking short passing', 'attacking volleys', 'defending', 'goalkeeping diving', 
           'goalkeeping handling', 'goalkeeping kicking', 'goalkeeping positioning', 'goalkeeping reflexes',  
           'ls', 'rs', 'lw', 'lf', 'rf', 'lam', 'ram', 'lcm', 'rcm', 'rm', 'ldm', 'rdm', 'rwb', 'lcb', 'rcb', 'rb',
          'body type', 'real face', 'player tags', 'potential', 'gk diving', 'gk handling', 'gk kicking', 
           'gk reflexes', 'gk speed', 'gk positioning', 'player traits', 'st', 'cf', 'rw', 'cam', 'lm', 'cm', 'lwb', 
          'lb', 'cb']
data.drop(to_drop, inplace=True, axis=1)

In [77]:
data['skill'] = data['skill dribbling'] + data['skill curve'] + data['skill fk accuracy'] + data['skill long passing'] + data['skill ball control']
data['movement'] = data['movement acceleration'] + data['movement sprint speed'] + data['movement agility'] + data['movement reactions'] + data['movement balance']
data['power'] = data['power shot power'] + data['power jumping'] + data['power stamina'] + data['power strength'] + data['power long shots']
data['mentality'] = data['mentality aggression'] + data['mentality interceptions'] + data['mentality positioning'] + data['mentality vision'] + data['mentality penalties'] + data['mentality composure']
data['defending'] = data['defending marking'] + data['defending standing tackle'] + data['defending sliding tackle']


to_drop2 = ['skill dribbling', 'skill curve', 'skill fk accuracy', 'skill long passing',
       'skill ball control', 'movement acceleration', 'movement sprint speed',
       'movement agility', 'movement reactions', 'movement balance', 'power shot power', 
        'power jumping', 'power stamina', 'power strength','power long shots', 'mentality aggression', 
        'mentality interceptions','mentality positioning', 'mentality vision', 'mentality penalties',
       'mentality composure', 'defending marking', 'defending standing tackle',
       'defending sliding tackle','team position', ]

data.drop(to_drop2, inplace=True, axis=1)


In [78]:
data.columns

Index(['age', 'height cm', 'weight kg', 'value eur', 'wage eur',
       'player positions', 'preferred foot', 'international reputation',
       'weak foot', 'skill moves', 'work rate', 'pace', 'shooting', 'passing',
       'dribbling', 'physic', 'cdm', 'skill', 'movement', 'power', 'mentality',
       'defending'],
      dtype='object')

In [79]:
#rename columns with a dictionary
corrected_column_names = {'height cm': 'height', 'weight kg': 'weight', 'value eur':'value', 
                          'wage eur': 'wage', 'preferred foot': 'foot', 
                          'international reputation': 'ir', 'physic':  'phy', 'shooting':'sho',
                        'passing': 'pas', 'dribbling': 'dri', 'weak foot': 'w/f', 'skill moves': 'sm',
                         'player positions': 'position', 'pace': 'pac'}
data.rename(columns = corrected_column_names, inplace = True)

In [80]:
data.columns

Index(['age', 'height', 'weight', 'value', 'wage', 'position', 'foot', 'ir',
       'w/f', 'sm', 'work rate', 'pac', 'sho', 'pas', 'dri', 'phy', 'cdm',
       'skill', 'movement', 'power', 'mentality', 'defending'],
      dtype='object')

In [81]:
#convert height and weight to units used in original data
data['height'] = data['height']*0.393701
data['weight'] = data['weight']*2.20462

In [82]:
#remove extra positions from position column
data['position'] = data['position'].apply(lambda x: x.split(',')[0])

#then encode with onehotencoder
pos = pd.DataFrame(data['position'])
encoded_position = bp_encoder.transform(pos).toarray()
onehot_encoded_pos = pd.DataFrame(encoded_position,columns=bp_encoder.categories_)

onehot_encoded_pos = onehot_encoded_pos.drop(['CB'], axis=1)

In [83]:
#encode foot column
foot = pd.DataFrame(data['foot'])
encoded_foot = foot_encoder.transform(foot).toarray()
onehot_encoded_foot = pd.DataFrame(encoded_foot,columns=foot_encoder.categories_)

onehot_encoded_foot = onehot_encoded_foot.drop(['Right'], axis=1)

In [84]:
#drop position and foot columns after encoding
data = pd.concat([data, onehot_encoded_pos, onehot_encoded_foot], axis=1)
data = data.drop(['position', 'foot'], axis =1)

In [85]:
data.columns

Index([      'age',    'height',    'weight',     'value',      'wage',
              'ir',       'w/f',        'sm', 'work rate',       'pac',
             'sho',       'pas',       'dri',       'phy',       'cdm',
           'skill',  'movement',     'power', 'mentality', 'defending',
          ('CAM',),    ('CDM',),     ('CF',),     ('CM',),     ('GK',),
           ('LB',),     ('LM',),     ('LW',),    ('LWB',),     ('RB',),
           ('RM',),     ('RW',),    ('RWB',),     ('ST',),   ('Left',)],
      dtype='object')

In [86]:
#split work rate into a/w and d/w
data['a/w'] = data['work rate'].apply(lambda x: x.split('/')[0])
data['d/w'] = data['work rate'].apply(lambda x: x.split('/')[1])
data.drop(['work rate'], inplace=True, axis=1)

#fill NAN to 'Medium'
data['a/w'].fillna(value='Medium', inplace=True)
data['d/w'].fillna(value='Medium', inplace=True)

#change to numerical values
labels = {'Low':0, 'Medium':1, 'High':2}
data['a/w'] = data['a/w'].replace(labels)
data['d/w'] = data['d/w'].replace(labels)  

In [87]:
data.columns

Index([      'age',    'height',    'weight',     'value',      'wage',
              'ir',       'w/f',        'sm',       'pac',       'sho',
             'pas',       'dri',       'phy',       'cdm',     'skill',
        'movement',     'power', 'mentality', 'defending',    ('CAM',),
          ('CDM',),     ('CF',),     ('CM',),     ('GK',),     ('LB',),
           ('LM',),     ('LW',),    ('LWB',),     ('RB',),     ('RM',),
           ('RW',),    ('RWB',),     ('ST',),   ('Left',),       'a/w',
             'd/w'],
      dtype='object')

In [97]:
data[''].describe()

count    881.000000
mean      64.871737
std        9.856269
min       34.000000
25%       59.000000
50%       66.000000
75%       72.000000
max       88.000000
Name: phy, dtype: float64

In [98]:
#remove + from 'cdm'
data['cdm'].fillna(value='50+6', inplace=True) #fill NaN with mean from train data set
data['cdm'] = data['cdm'].apply(lambda x: int(x.split('+')[0]) + int(x.split('+')[1]))

In [105]:
#fill in Nan values with mean
data['pac'].fillna(value=data['pac'].mean(), inplace=True)
data['sho'].fillna(value=data['sho'].mean(), inplace=True)
data['pas'].fillna(value=data['pas'].mean(), inplace=True)
data['phy'].fillna(value=data['phy'].mean(), inplace=True)
data['dri'].fillna(value=data['dri'].mean(), inplace=True)

In [114]:
#get predictions
predictions = lm.predict(data)

In [111]:
#check predictions against real scores
ova = [52, 62, 57, 59, 66, 65, 62, 73, 70, 62, 58, 72, 52, 68, 69, 67, 71, 61, 72, 70, 73, 64, 71, 56, 76, 56, 68, 63, 66, 67, 62, 70, 64, 62, 66, 70, 70, 66, 66, 70, 57, 55, 64, 64, 51, 58, 62, 60, 68, 67, 68, 71, 66, 56, 65, 60, 71, 72, 67, 75, 63, 67, 78, 66, 61, 69, 73, 64, 64, 67, 78, 64, 60, 58, 69, 68, 68, 82, 71, 54, 68, 70, 56, 69, 64, 62, 70, 69, 65, 79, 63, 61, 62, 63, 60, 63, 66, 71, 64, 68, 74, 71, 62, 65, 74, 57, 66, 71, 64, 64, 65, 65, 49, 75, 62, 67, 63, 75, 61, 69, 61, 72, 65, 60, 65, 61, 66, 67, 65, 70, 65, 64, 76, 76, 62, 67, 62, 70, 70, 60, 60, 68, 54, 71, 69, 72, 64, 66, 64, 67, 59, 59, 71, 56, 85, 79, 72, 65, 57, 73, 76, 67, 53, 66, 65, 67, 72, 75, 63, 70, 58, 66, 69, 88, 67, 62, 63, 72, 68, 69, 62, 64, 54, 75, 61, 58, 67, 77, 59, 49, 64, 68, 70, 72, 55, 62, 70, 77, 65, 81, 55, 61, 59, 70, 59, 64, 58, 64, 69, 63, 56, 58, 64, 67, 50, 67, 64, 51, 68, 56, 54, 64, 67, 60, 70, 78, 66, 72, 72, 66, 70, 76, 65, 62, 73, 68, 61, 72, 58, 56, 71, 60, 73, 65, 65, 74, 73, 67, 69, 65, 73, 69, 71, 74, 66, 74, 75, 79, 72, 69, 64, 66, 65, 72, 63, 65, 67, 69, 78, 51, 65, 75, 60, 67, 69, 64, 57, 63, 76, 64, 72, 59, 73, 65, 70, 80, 72, 59, 88, 61, 69, 61, 65, 69, 66, 70, 66, 60, 67, 68, 75, 62, 52, 69, 65, 81, 86, 72, 72, 66, 52, 56, 78, 62, 71, 78, 58, 60, 68, 64, 73, 53, 75, 68, 60, 77, 66, 73, 63, 71, 67, 65, 80, 77, 70, 66, 69, 65, 53, 52, 74, 65, 65, 67, 69, 70, 60, 59, 69, 68, 54, 82, 68, 75, 69, 71, 70, 79, 67, 58, 79, 64, 57, 68, 68, 66, 66, 73, 64, 81, 60, 69, 52, 59, 57, 68, 67, 55, 72, 76, 75, 64, 74, 65, 67, 59, 65, 66, 72, 73, 51, 67, 64, 62, 66, 68, 52, 56, 64, 70, 78, 71, 59, 69, 61, 62, 66, 64, 66, 67, 61, 72, 66, 64, 70, 70, 54, 74, 68, 64, 69, 65, 62, 76, 63, 66, 61, 72, 69, 76, 65, 76, 61, 52, 80, 67, 63, 60, 68, 66, 67, 59, 67, 72, 60, 51, 62, 81, 71, 69, 56, 67, 68, 69, 63, 65, 69, 62, 65, 71, 67, 66, 61, 73, 61, 51, 62, 61, 75, 65, 76, 68, 69, 65, 62, 64, 64, 73, 75, 71, 69, 58, 67, 60, 52, 65, 58, 77, 50, 80, 70, 68, 66, 69, 69, 60, 69, 61, 68, 80, 77, 67, 65, 74, 66, 65, 68, 78, 62, 80, 60, 87, 83, 68, 54, 67, 68, 60, 67, 61, 64, 59, 66, 73, 72, 58, 73, 72, 60, 53, 71, 65, 74, 73, 56, 75, 71, 64, 64, 66, 64, 70, 73, 78, 75, 53, 79, 73, 63, 67, 56, 73, 62, 54, 67, 63, 71, 69, 74, 74, 76, 68, 68, 64, 58, 62, 60, 63, 68, 68, 66, 75, 54, 70, 74, 62, 60, 67, 74, 73, 74, 55, 79, 67, 60, 68, 64, 50, 75, 63, 72, 57, 65, 66, 71, 59, 63, 57, 55, 68, 57, 67, 73, 52, 66, 68, 67, 56, 70, 69, 66, 63, 73, 65, 51, 61, 61, 78, 68, 65, 55, 64, 60, 62, 66, 67, 53, 67, 72, 64, 52, 65, 59, 70, 65, 79, 67, 75, 65, 61, 77, 63, 53, 61, 71, 69, 78, 48, 69, 63, 53, 67, 64, 76, 76, 60, 63, 66, 62, 67, 66, 67, 69, 68, 78, 62, 74, 72, 72, 65, 63, 59, 71, 68, 67, 70, 71, 65, 62, 58, 84, 68, 63, 62, 71, 68, 62, 78, 82, 67, 72, 79, 68, 69, 68, 60, 61, 76, 69, 72, 66, 68, 77, 62, 57, 66, 53, 62, 63, 63, 59, 74, 70, 72, 61, 66, 82, 69, 58, 70, 65, 69, 72, 67, 66, 76, 70, 71, 71, 66, 77, 59, 74, 68, 68, 83, 67, 63, 63, 69, 64, 67, 63, 64, 62, 61, 48, 72, 53, 49, 59, 77, 74, 67, 66, 60, 74, 58, 91, 71, 60, 84, 73, 68, 71, 66, 54, 65, 64, 64, 63, 69, 75, 71, 68, 62, 70, 65, 79, 57, 65, 65, 70, 66, 61, 61, 74, 59, 54, 59, 64, 75, 50, 66, 63, 69, 70, 66, 63, 57, 66, 76, 54, 73, 68, 62, 64, 62, 64, 67, 70, 75, 68, 57, 70, 64, 63, 51, 74, 80, 72, 65, 72, 53, 65, 77, 72, 63, 69, 57, 69, 65, 65, 67, 70, 79, 69, 55, 73, 66, 61, 77, 68, 66, 70, 73, 71, 70, 72, 66, 66, 64, 60, 67, 63, 58, 51, 61, 71, 65, 80, 75, 74, 64, 69, 62, 73, 65, 66, 72, 64, 68, 62, 56, 78, 78, 70, 73, 52, 68, 61, 72, 61, 60, 67, 69, 79, 66, 65, 76, 66, 74, 63, 71, 66, 71, 66, 54, 70, 71, 65, 64, 68, 66, 71, 66, 79, 64, 78, 64, 54, 70, 59, 59, 76, 70, 78, 65, 61, 68, 71, 63, 65, 67, 71, 64, 65, 62, 77, 48, 75, 67, 75, 68, 64, 67, 62, 64, 59, 64, 73, 58, 85, 63, 65, 62, 69, 72, 75, 59, 71, 55, 70, 70, 64, 66, 61, 64, 54, 72, 54, 66, 62, 63, 76, 69, 69, 73, 67, 74, 71, 56, 71, 63, 67, 68, 65, 80, 60, 58, 65, 75, 66, 70, 63, 69, 61, 69, 72, 67, 65, 67, 70, 72, 68, 67, 73, 72, 59, 68]

In [115]:
print('R2', metrics.r2_score(ova, predictions))
print('MSE', metrics.mean_squared_error(ova, predictions))
print('RMSE', np.sqrt(metrics.mean_squared_error(ova, predictions)))
print('RMSE', metrics.mean_absolute_error(ova, predictions))

R2 -377.7163096673413
MSE 17866.92619224565
RMSE 133.66722183185243
RMSE 127.2920344937877


In [ ]:
#really bad model :(